In [1]:
import piplite
await piplite.install("lib_rigid_body")
await piplite.install("widgetsnbextension")
await piplite.install("pythreejs")

from lib_rigid_body.rigid_body_FEM import *
import lib_rigid_body.rigid_body_FEM.bla as bla

import pythreejs as p3
from pythreejs import *

from IPython.display import display
import ipywidgets as widgets

import pickle

## Setup Objects

In [2]:
with open("box.pickle", "rb") as file:
    box = pickle.load(file)
# with open("cylinder.pickle", "rb") as file:
#     cylinder = pickle.load(file)

In [3]:
# set up physics simulation environment
rbs = RBS_FEM()
rbs.gravity = (0, 0.05, 0)

# set up physics simulation object for cube
r1 =  box
# r1.setPhat(3, 0.01)
# r1.setPhat(4, 0.01)

p1 = Transformation()
p1.setTranslation(0,0,0)
p1.setRotation(0,0,1)
p1.setRotation(1,1,1)
p1.setRotation(2,2,1)
r1.q=p1
c1 = rbs.addBody(r1)

r2 = box
# r2.setPhat(3, 0.05)
# r2.setPhat(4, 0.01)

p2 = Transformation()
p2.setTranslation(2,0,0)
p2.setRotation(0,0,1)
p2.setRotation(1,1,1)
p2.setRotation(2,2,1)
r2.q=p2

c2 = rbs.addBody(r2)
c2.pos = (0.5, 0.5, 0.5)
s = Spring(c1, c2, 10, 0.05)
rbs.addSpring(s)

f1 = rbs.addFix()
f1.pos = (-3, 3, -3)

s2 = Spring(f1, c1, 2, 0.05)
rbs.addSpring(s2)

rbs.saveState()

## Setup Graphics

In [4]:
def appendConnector(c,connectors):
    p = rbs.connectorPos(c)
    if(c.type == 0):
        color = 'green'
    else :
        color = 'black'
    connectors.append(
        p3.Mesh(p3.SphereBufferGeometry(0.2, 16, 16),
             p3.MeshStandardMaterial(color=color),
             position=(p[0], p[1], p[2])))

def initConnectors(l):
    connectors = []
    for s in l:
        cA = s.connectorA
        cB = s.connectorB
        appendConnector(cA,connectors);
        appendConnector(cB,connectors);
    return connectors
    

def updateConnectors(l, connectors):
    for i in range(len(l)):
        cA = l[i].connectorA
        cB = l[i].connectorB
        pA = rbs.connectorPos(cA);
        pB = rbs.connectorPos(cB);
        connectors[2*i].position = (pA[0], pA[1], pA[2])
        connectors[2*i+1].position = (pB[0], pB[1], pB[2])

def positionsOf(l):
    res = []
    for s in l:
        cA = s.connectorA
        cB = s.connectorB
        pA = rbs.connectorPos(cA);
        pB = rbs.connectorPos(cB);
        res.append ([ [pA[0], pA[1], pA[2]], [pB[0], pB[1], pB[2]] ])
    return res


connectorsSprings = initConnectors(rbs.springs())
springpos = positionsOf(rbs.springs())

if springpos:
    springgeo = p3.LineSegmentsGeometry(positions=springpos)
    m2 = p3.LineMaterial(linewidth=3, color='cyan')
    springs = p3.LineSegments2(springgeo, m2)

# connectorsBeams = initConnectors(rbs.beams())
# beampos = positionsOf(rbs.beams())

# if beampos:
#     beamgeo = p3.LineSegmentsGeometry(positions=beampos)
#     m2 = p3.LineMaterial(linewidth=4, color='blue')
#     beams = p3.LineSegments2(beamgeo, m2)


In [5]:
view_width = 1000
view_height = 700
buffergeos = []
p3meshes = []

# set up pythreejs 3d objects
for body in rbs.bodies():
    buffergeom = p3.BufferGeometry(attributes = {"position" : p3.BufferAttribute(body.vertices), "normal" : p3.BufferAttribute(body.normals)})
    # buffergeom.exec_three_obj_method("computeVertexNormals") # gives normals for shading
    material = p3.MeshPhongMaterial(color='#ff3333', shininess=150, morphTargets=True, side="DoubleSide")
    p3mesh = p3.Mesh(buffergeom, material, position=(0,0,0))
    buffergeos.append(buffergeom)
    p3meshes.append(p3mesh)

# extra scene contents
camera = p3.PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = p3.DirectionalLight(position=[0, 10, 10])
ambient_light = p3.AmbientLight()
grid = p3.GridHelper(500, 500//5, "#2F4F4F", "#2F4F4F")
axesHelper = p3.AxesHelper(5)

# set up scene
scene = p3.Scene(children=[camera, key_light, ambient_light, grid, axesHelper, *p3meshes, *connectorsSprings] + ([] if not springpos else [springs]))
# scene.add(p3mesh)
controller = p3.OrbitControls(controlling=camera)
renderer = p3.Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height, antialias=True) # if performance is bad, try removing antalias=True

In [6]:
# play/reset button
play = widgets.Play(
    value=0,
    min=0,
    max=10000,
    step=1,
    interval=100/30,
    description="Press play",
    disabled=False
)

In [7]:
for m in p3meshes:
    m.matrixAutoUpdate = False # make mesh movable

def refresh():
    "updates all pythreejs object transformations"
    for i in range(0, len(rbs.bodies())):
        p3meshes[i].matrix=rbs.bodies()[i].q.asTuple()
    updateConnectors(rbs.springs(),connectorsSprings)
    springpos = positionsOf(rbs.springs())
    if springpos:
        springs.geometry = p3.LineSegmentsGeometry(positions=springpos)
    
    # updateConnectors(rbs.beams(),connectorsBeams)
    # beampos = positionsOf(rbs.beams())
    # if beampos:
    #     beams.geometry = p3.LineSegmentsGeometry(positions=beampos)

def update():
    "update function, gets called every timestep; quasi main loop"
    simulate(rbs, 1, 20)
    refresh()

def observer(state):
    "event handler for clickable buttons"
    # if there is a change in time
    if state["name"] == "value":
        # it might be a reset
        if str(state["new"]) == "0":
            rbs.reset()
            refresh()
        # or it might be a progress in time
        else:
            update()
    # repeat is used as an alias to reset
    elif state["name"] == "repeat":
        rbs.reset()
        refresh()

play.observe(observer)

## Renderer

In [8]:
display(widgets.HBox([play, widgets.HTML("<b>click-and-drag to rotate, scroll to zoom, right-click-and-drag to move<b>")]))
display(renderer)
refresh()

Renderer(camera=PerspectiveCamera(aspect=1.4285714285714286, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0…